**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video opencv-python

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten


import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = ""
# due to GPU issue on my computer 

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

During training, the function act uses an hyperparameter $\epsilon$ between 0 and 1 to balance choosing an action by using the policy learned, or randomly. It is used to balance exploitation of the known policy, and exploration of other policies in order to avoid local optimas.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=51
epochs_test=11

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

the matrix position incorporates the information on where the agent is (=1), where it can go (=0), and can't (-1). the other matrix board is for rewards for each coordinates : +0.5, 0 , or -1

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0,4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            #loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 5.5/6.0. Average score (-0.5)
Win/lose count 8.5/16.0. Average score (-4.0)
Win/lose count 10.5/16.0. Average score (-4.5)
Win/lose count 7.0/16.0. Average score (-5.625)
Win/lose count 13.5/20.0. Average score (-5.8)
Win/lose count 8.5/12.0. Average score (-5.416666666666667)
Win/lose count 9.5/21.0. Average score (-6.285714285714286)
Win/lose count 5.0/15.0. Average score (-6.75)
Win/lose count 6.5/9.0. Average score (-6.277777777777778)
Win/lose count 9.0/12.0. Average score (-5.95)
Win/lose count 8.0/15.0. Average score (-6.045454545454546)
Final score: -6.045454545454546


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




#### Answer 
Starting from the definition introduced at the beginning of this homework
\begin{align} Q^\pi(s,a) &= \mathbf{E}\left[\sum_t \gamma^{t}r(s_t,a_t)|s_0=s,a_0=a\right] \\
&= r(s, a) + \gamma \mathbf{E}\left[\sum\limits_{t \ge 1} \gamma^{t - 1} r(s_t,a_t)|s_0=s,a_0=a\right] \\
&= r(s, a) + \gamma \sum\limits_{s'} {P}(s_1 = s' | s_0 = s) \mathbf{E}\left[\sum\limits_{t \ge 1} \gamma^{t - 1} r(s_t,a_t)|s_1=s',a_1=a'=\pi(s)\right] \\
&= r(s, a) + \gamma \sum\limits_{s'} {P}(s_1 = s' | s_0 = s) Q^\pi(s', a') \\
&= \mathbf{E}_{(s',a')\sim p(.|s,a)}\left[r(s,a)+\gamma Q^{\pi}(s',a')\right] \end{align}
We obtain the recursive formulation of the Q value based on the optimal policy $\pi^{*}$, and we can proceed to compute the general Q value as follows, by propagating the max :
\begin{align} Q^{*}(s, a) &= \max\limits_{\pi}  Q^{\pi}(s, a) \\ 
&= \max\limits_{\pi} \left[ r(a, s) + \gamma \sum_{s'} {P}(s_1 = s' | s_0 = s) Q^{\pi^*}(s', \pi(s)) \right] \\ &= r(a, s) + \gamma \sum\limits_{s'} {P}(s_1 = s' | s_0 = s) \max\limits_{\pi} Q^{\pi^*}(s', \pi(s)) \\ &= r(a, s) + \gamma \sum_{s'} {P}(s_1 = s' | s_0 = s) \max\limits_{a'} Q^{*}(s', a') \\ &= \mathbf{E}_{s'\sim \pi^(.|s,a)}\left[r(s,a)+\gamma\max\limits_{a'}Q^{*}(s',a')\right] \end{align}
In order to learn this function, we must have a loss function. From the previous formulation we can get
\begin{align}
    & \mathbf{E}_{s'\sim \pi^(.|s,a)}\left[r(s,a)+\gamma\max\limits_{a'}Q^{*}(s',a')\right]-Q^{*}(s,a) &=0\\
    \implies & \mathbf{E}_{s'\sim \pi^(.|s,a)}\left[r(s,a)+\gamma\max\limits_{a'}Q^{*}(s',a')-Q^{*}(s,a)\right] &=0
\end{align}
And thus estimating it using the $L_2$ distance as loss function to minimize is a plausible objective.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
          self.memory = self.memory[1:]

    def random_access(self):
        return np.random.permutation(self.memory)

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        #print("learned_act",s.shape)
        s2 = self.model.predict(s[np.newaxis,:,:,:])
        #print("learned_act2",s.shape)
        #print(s2)
        a = np.argmax(s2,axis=1)[0]
        #print("learned_act3",a)
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        batch = self.memory.random_access()[:self.batch_size]

        
        for i in range(self.batch_size):
            if len(batch)<=i:
              break
            s_i, n_s_i, a_i, r_i, game_over_i = batch[i]
            #print("a_i",a_i)
            #print("game_",game_over_i.shape)
            #print("r_i",r_i.shape)
            input_states[i] = s_i 
            #default
            target_q[i] = self.model.predict(s_i[np.newaxis,:,:,:])
            
            if game_over_i:
                target_q[i,a_i] = r_i
            else:
                target_q[i,a_i] = r_i + self.discount * np.max(self.model.predict(s_i[np.newaxis,:,:,:]))
        #print("input_shape",input_states.shape)
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,2)))
        model.add(Dense(100))
        model.add(Activation('relu'))
        model.add(Dense(30))
        model.add(Activation('relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 000/051 | Loss 0.0091 | Win/lose count 3.5/7.0 (-3.5)
Epoch 001/051 | Loss 0.0068 | Win/lose count 3.0/5.0 (-2.0)
Epoch 002/051 | Loss 0.0161 | Win/lose count 3.5/8.0 (-4.5)
Epoch 003/051 | Loss 0.0066 | Win/lose count 8.0/7.0 (1.0)
Epoch 004/051 | Loss 0.0035 | Win/lose count 4.5/4.0 (0.5)
Epoch 005/051 | Loss 0.0097 | Win/lose count 7.0/2.0 (5.0)
Epoch 006/051 | Loss 0.0035 | Win/lose count 2.0/1.0 (1.0)
Epoch 007/051 | Loss 0.0022 | Win/lose count 6.0/3.0 (3.0)
Epoch 008/051 | Loss 0.0030 | Win/lose count 5.5/7.0 (-1.5)
Epoch 009/051 | Loss 0.0024 | Win/lose count 7.0/7.0 (0.0)
Epoch 010/051 | Loss 0.0032 | Win/lose count 6.0/4.0 (2.0)
Epoch 011/051 | Loss 0.0096 | Win/lose count 4.0/3.0 (1.0)
Epoch 012/051 | Loss 0.0113 | Win/lose count 6.5/0 (6.5)
Epoch 013/051 | Loss 0.0076 | Win/lose count 5.0/2.0 (3.0)
Epoch 014/051 | Loss 0.0020 | Win/lose count 7.0/5.0 (2.0)
Epoch 015/051 | Loss 0.0054 | W

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        
        model = Sequential()
        model.add(Conv2D(filters=30,kernel_size=3,input_shape=(5,5,self.n_state)))
        model.add(Activation('relu'))
        model.add(Conv2D(filters=20,kernel_size=3))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

Epoch 000/051 | Loss 0.0056 | Win/lose count 4.0/2.0 (2.0)
Epoch 001/051 | Loss 0.0085 | Win/lose count 2.0/7.0 (-5.0)
Epoch 002/051 | Loss 0.0019 | Win/lose count 3.0/6.0 (-3.0)
Epoch 003/051 | Loss 0.0026 | Win/lose count 3.0/2.0 (1.0)
Epoch 004/051 | Loss 0.0001 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/051 | Loss 0.0000 | Win/lose count 2.0/5.0 (-3.0)
Epoch 006/051 | Loss 0.0102 | Win/lose count 1.0/4.0 (-3.0)
Epoch 007/051 | Loss 0.0161 | Win/lose count 2.5/3.0 (-0.5)
Epoch 008/051 | Loss 0.0021 | Win/lose count 1.5/6.0 (-4.5)
Epoch 009/051 | Loss 0.0203 | Win/lose count 2.0/7.0 (-5.0)
Epoch 010/051 | Loss 0.0176 | Win/lose count 3.0/5.0 (-2.0)
Epoch 011/051 | Loss 0.0074 | Win/lose count 4.0/5.0 (-1.0)
Epoch 012/051 | Loss 0.0039 | Win/lose count 6.5/4.0 (2.5)
Epoch 013/051 | Loss 0.0049 | Win/lose count 4.5/4.0 (0.5)
Epoch 014/051 | Loss 0.0081 | Win/lose count 3.5/2.0 (1.5)
Epoch 015/051 | Loss 0.0168 | Win/lose count 3.0/2.0 (1.0)
Epoch 016/051 | Loss 0.0021 | Win/lose count 4.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [22]:
env = Environment(grid_size=size, max_time=T,temperature=0.8)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 27.5/1.0. Average score (26.5)
Win/lose count 22.0/3.0. Average score (22.75)
Win/lose count 28.5/4.0. Average score (23.333333333333332)
Win/lose count 12.0/4.0. Average score (19.5)
Win/lose count 36.5/3.0. Average score (22.3)
Win/lose count 22.5/2.0. Average score (22.0)
Win/lose count 12.5/1.0. Average score (20.5)
Win/lose count 14.5/5.0. Average score (19.125)
Win/lose count 5.0/1.0. Average score (17.444444444444443)
Win/lose count 34.5/5.0. Average score (18.65)
Win/lose count 8.0/1.0. Average score (17.59090909090909)
Final score: 17.59090909090909
Test of the FC
Win/lose count 5.0/0. Average score (5.0)
Win/lose count 9.0/3.0. Average score (5.5)
Win/lose count 3.5/1.0. Average score (4.5)
Win/lose count 4.0/0. Average score (4.375)
Win/lose count 2.5/1.0. Average score (3.8)
Win/lose count 3.5/1.0. Average score (3.5833333333333335)
Win/lose count 10.5/2.0. Average score (4.285714285714286)
Win/lose count 3.5/0. Average score (4.1875)
Win/lose

In [17]:
HTML(display_videos('cnn_test10.mp4'))

In [18]:
HTML(display_videos('fc_test10.mp4'))

The issue is that the agent does not explore and generally stays very close to its starting point, since it does not remember where it has already been, it turns around. When the temperature is high, and the starting location not too bad, it explores well due to being surrounded by bonuses. But when temperature is lower and bonuses sparser, it does only clear the area around its starting point.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [19]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        agent.set_epsilon(agent.epsilon*0.9)
        print("New Epsilon = ",agent.epsilon)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        #reward = self.board[self.x, self.y]
        game_over = self.t > self.max_time
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        #reset also the malus
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    


In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

New Epsilon =  0.81
Epoch 000/051 | Loss 0.0200 | Win/lose count 12.5/33.60000000000012 (-21.100000000000122)
New Epsilon =  0.7290000000000001
Epoch 001/051 | Loss 0.0136 | Win/lose count 12.0/24.00000000000007 (-12.000000000000071)
New Epsilon =  0.6561000000000001
Epoch 002/051 | Loss 0.0049 | Win/lose count 10.5/24.400000000000087 (-13.900000000000087)
New Epsilon =  0.5904900000000002
Epoch 003/051 | Loss 0.0161 | Win/lose count 9.0/24.600000000000048 (-15.600000000000048)
New Epsilon =  0.5314410000000002
Epoch 004/051 | Loss 0.0090 | Win/lose count 19.5/20.100000000000023 (-0.6000000000000227)
New Epsilon =  0.47829690000000014
Epoch 005/051 | Loss 0.0032 | Win/lose count 12.0/23.700000000000088 (-11.700000000000088)
New Epsilon =  0.43046721000000016
Epoch 006/051 | Loss 0.0034 | Win/lose count 11.0/24.60000000000002 (-13.60000000000002)
New Epsilon =  0.38742048900000015
Epoch 007/051 | Loss 0.0326 | Win/lose count 9.0/24.500000000000068 (-15.500000000000068)
New Epsilon =  0.

In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 1.5/19.0. Average score (-17.5)
Win/lose count 7.0/17.69999999999998. Average score (-14.09999999999999)
Win/lose count 7.5/17.799999999999983. Average score (-12.833333333333321)
Win/lose count 9.0/16.399999999999963. Average score (-11.474999999999982)
Win/lose count 11.5/16.19999999999996. Average score (-10.119999999999978)
Win/lose count 11.5/15.199999999999962. Average score (-9.049999999999974)
Win/lose count 17.0/15.099999999999962. Average score (-7.485714285714258)
Win/lose count 4.0/18.099999999999987. Average score (-8.312499999999975)
Win/lose count 5.5/18.29999999999999. Average score (-8.811111111111087)
Win/lose count 13.5/14.199999999999966. Average score (-7.999999999999974)
Win/lose count 9.0/17.69999999999998. Average score (-8.06363636363634)
Final score: -8.06363636363634


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***